In [1]:
from lib import util, analyze

In [2]:
occupation_categories = ('Salaried/Employed (Private/Public Sector)', 'Agriculturalists (Farmers)', 'Laborers/Skilled Workers', 'Students',  'Homemakers', 'Government Employees (Civil Services, Defense, Police)', 'Self-employed Professionals (Doctors, Lawyers, Chartered Accountants)', 'Retired/Pensioners', 'Business Owners/Entrepreneurs', 'Freelancers/Consultants', 'Unemployed/Jobseekers')
occupation_stats = (12.6, 23, 11.4, 19.2, 14.7, 3.2, 3.4, 6, 2.6, 2, 1.9)
education_categories = ('Primary or less', 'Secondary', 'Tertiary or more')
education_stats = (69.4, 16.4, 14.2)
vpn_stats = (8, 4, 0, 70)
sim_card_changes_stats = (1.2, 0.8, 0, 60)
trading_account_stats = (1.27, 0.9, 0, 10)
recharge_frequency_stats = (1.426, 0.57, 0, 20)
partener_categories = ('Never Married', 'Currently Married', 'Widowed', 'Divorced')
partener_stats = (0.39, 0.475, 0.13, 0.005)
dependents_stats = (2.25, 1.7, 0, 10)
streaming_services_number_stats = (2.5, 1, 0, 10)
streaming_services_delay_stats = (1.2, 1.4, 0, 6)
family_defaulters_stats = (1.81, 1.62, 0, 10)
smart_cards_stats = (2.6, 0.9, 0, 6)
medical_bills_stats = (9000, 5000, 0, 50000)
default_gst_filing_stats = (0.39,0.65,0,3)
antivirus_subscriptions_stats = (0.81, 0.53, 0, 10)
betting_categories = ('Yes', 'No')
betting_stats = (20.69, 79.31)
truecaller_categories = ('Red', 'Blue', 'Golden')
truecaller_stats = (0.2, 0.75, 0.5)
shopping_behavior_stats = (0.35, 0.7, 0, 25)
utility_bills_stats = (9500, 6500, 2500, 35000)
registered_vehicles_stats = (0.3, 0.5, 0, 10)
registered_vehicles_challans_stats = (0.12, 0.5, 0, 100)
certificates_stats = (0.4, 0.6, 0, 10)
licenses_stats = (0.65, 0.72, 0, 7)
insurances_stats = (0.75, 1.2, 0, 7)
fd_dissolved_stats = (0.28, 0.65, 0, 10)
international_transactions_stats = (0.45, 13.31, 0, 1000)
public_memberships_stats = (0.035, 0.2, 0, 7)
reviews_categories = ('1 Star', '2 Star', '3 Star', '4 Star', '5 Star')
reviews_stats = (4, 6, 5, 45, 40)
pet_bills_stats = (26000, 30000, 0, 250000)
accounts_stats = (2.05, 1.5, 0, 8)
social_sentiments_categories = ('Strongly Negative', 'Negative', 'Neutral', 'Positive', 'Strongly Positive')
social_sentiments_stats = (20, 32, 32, 12, 4)
upi_failures_stats = (25, 15, 0, 200)

In [3]:
data = util.fabricate_base_data()

In [4]:
data['Occupation'] = util.create_categorical_distribution(occupation_categories, occupation_stats, nan_probability=0.01)
data['Education'] = util.create_categorical_distribution(education_categories, education_stats, nan_probability=0.01)
data['VPN Usage (hours per week)'] = util.create_truncated_norm_distribution(vpn_stats, nan_probability=0.01)
data['SIM Card Changes'] = util.create_truncated_norm_distribution(sim_card_changes_stats, precision=0, nan_probability=0.01)
data['Trading Accounts'] = util.create_truncated_norm_distribution(trading_account_stats, precision=0, nan_probability=0.01)
data['Recharge Frequency'] = util.create_truncated_norm_distribution(recharge_frequency_stats, precision=0,nan_probability=0.01)
data['Partner'] = util.create_categorical_distribution(partener_categories, partener_stats, nan_probability=0.01)
data['Dependents'] = util.create_truncated_norm_distribution(dependents_stats, precision=0, nan_probability=0.01)
data['Streaming Services'] = util.create_truncated_norm_distribution(streaming_services_number_stats, precision=0, nan_probability=0.01)
data['Payment Delay of Streaming Services (per year)'] = util.create_truncated_norm_distribution(streaming_services_delay_stats, precision=0, nan_probability=0.01)
data['Family Defaulters'] = util.create_truncated_norm_distribution(family_defaulters_stats, precision=0, nan_probability=0.01)
data['Smart Cards'] = util.create_truncated_norm_distribution(smart_cards_stats, precision=0, nan_probability=0.01)
data['Medical Bills'] = util.create_truncated_norm_distribution(medical_bills_stats, precision=1, nan_probability=0.01)
data['Default in GST filing (per quarter)'] = util.create_truncated_norm_distribution(default_gst_filing_stats, precision=0, nan_probability=0.01)
data['Antivirus Subscriptions'] = util.create_truncated_norm_distribution(antivirus_subscriptions_stats, precision=0, nan_probability=0.01)
data['Betting Apps'] = util.create_categorical_distribution(betting_categories, betting_stats, nan_probability=0.01)
data['TrueCaller Flag'] = util.create_categorical_distribution(truecaller_categories, truecaller_stats, nan_probability=0.01)
data['Shopping Behavior (returns per month)'] = util.create_truncated_norm_distribution(shopping_behavior_stats, precision=0, nan_probability=0.01)
data['Utility Bills (per month)'] = util.create_truncated_norm_distribution(utility_bills_stats, precision=0, nan_probability=0.01)
data['Registered Vehicles'] = util.create_truncated_norm_distribution(registered_vehicles_stats, precision=0, nan_probability=0.01)
data['Registered Vehicles Challan (per year)'] = util.create_truncated_norm_distribution(registered_vehicles_challans_stats, precision=0, nan_probability=0.01)
data['Certificates'] = util.create_truncated_norm_distribution(certificates_stats, precision=0, nan_probability=0.01)
data['Licenses'] = util.create_truncated_norm_distribution(licenses_stats, precision=0, nan_probability=0.01)
data['Insurances'] = util.create_truncated_norm_distribution(insurances_stats, precision=0, nan_probability=0.01)
data['FDs Dissolved'] = util.create_truncated_norm_distribution(fd_dissolved_stats, precision=0, nan_probability=0.01)
data['International Transactions'] = util.create_truncated_norm_distribution(international_transactions_stats, precision=0, nan_probability=0.01)
data['Public Memberships'] = util.create_truncated_norm_distribution(public_memberships_stats, precision=0, nan_probability=0.01)
data['Reviews received'] = util.create_categorical_distribution(reviews_categories, reviews_stats, nan_probability=0.01)
data['Pet Bills (per year)'] = util.create_truncated_norm_distribution(pet_bills_stats, precision=1, nan_probability=0.01)
data['Accounts'] = util.create_truncated_norm_distribution(accounts_stats, precision=0, nan_probability=0.01)
data['Sentiment on Social Media'] = util.create_categorical_distribution(social_sentiments_categories, social_sentiments_stats, nan_probability=0.01)
data['UPI Failures'] = util.create_truncated_norm_distribution(upi_failures_stats, precision=0, nan_probability=0.01)

In [ ]:
analyze.analyze_dataframe(data)

Analyzing DataFrame...
Shape: (10000, 38)
Data types: 
Name                                               object
Age                                               float64
Gender                                             object
City                                               object
E-mail                                             object
Phone No.                                          object
Occupation                                         object
Education                                          object
VPN Usage (hours per week)                        float64
SIM Card Changes                                  float64
Trading Accounts                                  float64
Recharge Frequency                                float64
Partner                                            object
Dependents                                        float64
Streaming Services                                float64
Payment Delay of Streaming Services (per year)    float64
Family Defaulters

Pairplot of numerical features:


<Figure size 10000x6000 with 0 Axes>